# Population Generations Story

**What this notebook shows**
- Geospatial analysis with GeoPandas
- Live OpenStreetMap queries (Overpass)
- Interactive visualization with Altair

**Data**
- Remote sources: https://gist.githubusercontent.com/TieJean/40e9ccc69f0cc65b8e06ccf3fd60a3f0/raw/04ec5f998e63185e65f4d415827f1838ff8a25ab/austin.geojson, https://gist.githubusercontent.com/TieJean/4e9f595eb3cb5dc60a960c3a499f6141/raw/171118ec54c599ce4bf9a30f9ab176427a47629f/austin_schools.geojson (+1 more)

##### Framing the exploration
We are using historical US Census data from Vega Datasets to study how generational cohorts ebb and flow over time. The goal is to keep a reusable reference for common transformations (e.g., deriving generations, toggling encodings, animating comparisons) that we can remix in future storytelling work.


In [ ]:
from importlib import import_module

for package in ("altair", "pandas"):
    import_module(package)

print("Altair and pandas imports verified; proceeding with population deep dive.")


In [ ]:
# Import the necessary libraries and data
import altair as alt
import pandas as pd
from vega_datasets import data as vega_data

df_pop = vega_data.population()

In [ ]:
df_pop.head()

In [ ]:
df_pop.shape

In [ ]:
df_pop.describe()

In [ ]:
df_pop.info()

##### Exploration 1 – Add in the "Boomer" label

As we can see from inspecting the dataframe, our data only gives us information for:
  - year
  - age
  - sex
  - people
  
But, we want to be able to highlight just the people born between 1946 - 1964 as a separate group.  To accomplish this, we want to create a new categorical attribute - `Generation`

Using pandas data manipulation techniques, add a new column to `df_pop` named `Generation` that either has the value `Baby Boomer` or `Other`.

In [ ]:
# your code here

def label_generation(row):
    birth_year = row['year'] - row['age']
    if 1946 <= birth_year <= 1964:
        return 'Baby Boomer'
    else:
        return 'Other'

df_pop['Generation'] = df_pop.apply(label_generation, axis=1)
df_pop.head()

##### Exploration 2 – Change the encoding for `sex`

As in our earlier prototype work, the sex "Male" is encoded as the number `1` and "Female" is encoded as `2`.  Modify the dataframe  `df_pop` to replace the encoding with the string so when we create our plots this will automatically have the legend come out correctly (note, you can map numbers to labels in Altair as well).  

In [ ]:
# your code here
df_pop['sex'] = df_pop['sex'].map({1: 'Male', 2: 'Female'})
df_pop.head()

In [ ]:
df_pop.head()

##### Exploration 3 – Show the Population Change Over Time with a Slider

Now, we have a snapshot of 2 different years next to each other, but what about creating a crude animation by controlling the the year displayed with a slider?

Create a slider using [this example](https://altair-viz.github.io/gallery/us_population_over_time.html) to help guide you.  Our plot will look similar, except we have not split our bar chart up by `sex` yet. Name the slider 'Select Year:' (this in controlled in `binding_range`, and not in the `selection_single` parameters).  

Encode membership of the Baby Boomer generation with color using "#7D3C98" (purple) for the boomers "#F4D03F" (gold) for other.

Start the slider at 1900.

In [ ]:
# your code here
year_slider = alt.binding_range(min=1900, max=2020, step=1, name='Select Year:')
year_select = alt.selection_point(fields=['year'], bind=year_slider, value=1900)

bar_chart = alt.Chart(df_pop).mark_bar().encode(
    x=alt.X('age:O', title='Age'),
    y=alt.Y('sum(people):Q', title='Population'),
    color=alt.Color('Generation:N',
                    scale=alt.Scale(domain=['Baby Boomer', 'Other'],
                                   range=['#7D3C98', '#F4D03F'])),
    tooltip=['age:O', 'sum(people):Q', 'Generation:N']
).add_params(
    year_select
).transform_filter(
    year_select
).properties(
    width=600,
    height=400,
    title='Population by Age and Generation Over Time'
)

bar_chart

##### Exploration 4 – Linking

Let us take a closer look at just the year 2000 data, and find what the distribution of sex is for each individual age grouping.  Plot the distribution of ages as a bar chart for just the year 2000, and link a histogram that will plot the distribution of sex for the current selection.  It should default to no age group selected (as in, all bars are colored, meaning none are selected).  The histogram for the sex distribution should appear below the year 2000 data (vertically concatenated). When a bar on the top chart is selected, indicate its selection by turning the other bars light gray.  The histogram of the sex distribution below it should be a horizontal bar chart.

Encode membership of the Baby Boomer generation with color using "#7D3C98" (purple) for the boomers, and "#F4D03F" (gold) for other.

In [ ]:
# your code here
df_2000 = df_pop[df_pop['year'] == 2000]

age_select = alt.selection_point(fields=['age'], empty=True)

age_chart = alt.Chart(df_2000).mark_bar().encode(
    x=alt.X('age:O', title='Age'),
    y=alt.Y('sum(people):Q', title='Population'),
    color=alt.condition(
        age_select,
        alt.Color('Generation:N',
                  scale=alt.Scale(domain=['Baby Boomer', 'Other'],
                                 range=['#7D3C98', '#F4D03F'])),
        alt.value('lightgray')
    ),
    tooltip=['age:O', 'sum(people):Q', 'Generation:N']
).add_params(
    age_select
).properties(
    width=600,
    height=300,
    title='Distribution of Ages for Year 2000'
)

sex_chart = alt.Chart(df_2000).mark_bar().encode(
    y=alt.Y('sex:N', title='Sex'),
    x=alt.X('sum(people):Q', title='Population'),
    color=alt.Color('Generation:N',
                    scale=alt.Scale(domain=['Baby Boomer', 'Other'],
                                   range=['#7D3C98', '#F4D03F'])),
    tooltip=['sex:N', 'sum(people):Q', 'Generation:N']
).transform_filter(
    age_select
).properties(
    width=600,
    height=150,
    title='Sex Distribution for Selected Age Group'
)

linked_chart = age_chart & sex_chart
linked_chart

##### Exploration 5 – Combine Q3 and Q4 to One Chart

In Q4, we linked the distribution of sex to the age selection for just the year 2000.  Let us visualize all the data by incorporating the year selection slider from Q3 so that you can select which year of data you are viewing. Retain the ability to just select one age group for the sex distribution, and default to no age group selected.

Add a tooltip so you can see exactly how many people are in the age range for the top "Distribution of Ages for the Selected Year" histogram.

Encode membership of the Baby Boomer generation with color using "#7D3C98" (purple) for the boomers and "#F4D03F" (gold) for other.

In [ ]:
# your code here
year_slider = alt.binding_range(min=1900, max=2020, step=1, name='Select Year:')
year_select = alt.selection_point(fields=['year'], bind=year_slider, value=1900)

age_select = alt.selection_point(fields=['age'], empty=True)

age_chart = alt.Chart(df_pop).mark_bar().encode(
    x=alt.X('age:O', title='Age'),
    y=alt.Y('sum(people):Q', title='Population'),
    color=alt.condition(
        age_select,
        alt.Color('Generation:N',
                  scale=alt.Scale(domain=['Baby Boomer', 'Other'],
                                 range=['#7D3C98', '#F4D03F'])),
        alt.value('lightgray')
    ),
    tooltip=['age:O', 'sum(people):Q', 'Generation:N']
).add_params(
    year_select,
    age_select
).transform_filter(
    year_select
).properties(
    width=600,
    height=300,
    title='Distribution of Ages for the Selected Year'
)

sex_chart = alt.Chart(df_pop).mark_bar().encode(
    y=alt.Y('sex:N', title='Sex'),
    x=alt.X('sum(people):Q', title='Population'),
    color=alt.Color('Generation:N',
                    scale=alt.Scale(domain=['Baby Boomer', 'Other'],
                                   range=['#7D3C98', '#F4D03F'])),
    tooltip=['sex:N', 'sum(people):Q', 'Generation:N']
).transform_filter(
    year_select
).transform_filter(
    age_select
).properties(
    width=600,
    height=150,
    title='Sex Distribution for Selected Age Group'
)

# Vertically concatenate the charts
combined_chart = age_chart & sex_chart
combined_chart

Geoviz notes focus on mapping workflows we deploy during client engagements: pulling in OpenStreetMap features, blending GeoNames exports, and styling overlays directly in Altair. It is only a teaser of what we typically build, but it keeps the essentials close at hand.


In [ ]:
import altair as alt
import pandas as pd

### Install packages

For this tutorial we will continue to rely on Altair and Pandas, but add **GeoPandas**, which will help us to work with DataFrames that contain spatial entities to carry out geometric analysis on them. As before, the pip install command is carried out via the shell, which is indicated by the exclamation mark at the beginning of the line:

In [ ]:
%pip install geopandas
import geopandas as gpd

To access the data of the OpenStreetMap, we will install the handy package **OSMPythonTools**:

In [ ]:
%pip install --upgrade OSMPythonTools

Once we have the tools assembled, we can get started working with geospatial data. There are actually plenty of formats used to record geospatial data, but GeoJSON has become an important standard for exchanging geospatial data on the web. However, please note that GeoPandas can actually load many other vector-based data formats used in digital cartography, such as shapefiles and GeoPackage.

### Import GeoJSON

Suppose we would like to get the geographic boundaries of Austin's neighborhoods. Akin to how we would read a JSON file with Pandas, we can also use `read_file()` provided by GeoPandas simply by passing the geojson filename and get a geographic DataFrame back:

In [ ]:
neighborhoods = gpd.read_file("https://gist.githubusercontent.com/TieJean/40e9ccc69f0cc65b8e06ccf3fd60a3f0/raw/04ec5f998e63185e65f4d415827f1838ff8a25ab/austin.geojson")

The main difference between a regular Pandas DataFrame is that a GeoDataFrame features a `geometry` column, which is a geoseries containing the points, paths, and polygons for each row. For example, if each row represents one neighborhood, the respective geometries would probably contain the geospatial boundaries…

💡 *Are you curious what the neighborhoods dataframe actually looks like? Take a look at it with the methods you know by now:*

In [ ]:
neighborhoods.head()

Geographically speaking, the districts are defined by their geographic shapes, which are represented as polygons, each of which is a list of tuples of geographic coordinates. Next we add information about schools in Austin:

In [ ]:
schools = gpd.read_file("https://gist.githubusercontent.com/TieJean/4e9f595eb3cb5dc60a960c3a499f6141/raw/171118ec54c599ce4bf9a30f9ab176427a47629f/austin_schools.geojson")
schools.head()

💡 *Have a look at the schools as well, and compare the contents of the `geometry` columns in schools and districts. Do you notice anything?*

### Query OpenStreetMap

OpenStreetMap (OSM) is "a collaborative project to create a free editable map of the world". As such it has millions of contributing users who have been collecting, updating and refining map data for over 15 years, which has generated a vastly comprehensive source of geographic data. It is by no means complete—whatever this would mean—but it is an impressively large geographic database and, of course, a map in itself, too.

OpenStreetMap has its own kind of query language, which is quite compact and can also be a source for errors. To make query formulation easier, you can either use the web interface [overpass turbo](http://overpass-turbo.eu) or the `overpassQueryBuilder`, which provides access to the main parameters:

In [ ]:
from OSMPythonTools.overpass import overpassQueryBuilder

AustinAreaID = 3600000000 + 113314
# The area ID of a city is found by adding 3600000000 to the city's relation ID
# You can look up the relation ID of any city by searching on the OSM website; for example, https://www.openstreetmap.org/relation/113314

library_query = overpassQueryBuilder(
    area=AustinAreaID, # the query can be constrained by an area of an item
    elementType='node', # which are points (OSM also has ways and relations)
    # the selector in the next line is really the heart of the query:
    selector='"amenity"="library"', # we're looking for libraries
    out='body', # body indicates that we want the data, not just the count
    includeGeometry=True # and we want the geometric information, too
)

library_query

The output of above cell is the compact version of the query, which is carried out in the next step:



In [ ]:
from OSMPythonTools.overpass import Overpass
overpass = Overpass()

lib_data = overpass.query(library_query)

The variable `lib_data` now already contains the result from the query against OSM. Let's have a look at it. With `nodes()` we can access the retrieved points. Let's take a look at the first entry:

In [ ]:
lib_data.nodes()[0].tags()

Similarly, we can also access the geometry, which in this case is just a point:

In [ ]:
lib_data.nodes()[0].geometry()

Next, we use the compact form of a list comprehension to extract the libraries' names and coordinates:


In [ ]:
libraries = [ (lib.tag("name"), lib.geometry() ) for lib in lib_data.nodes()]

… which we turn into a GeoDataFrame. By naming the second column `geometry` we indicate towards GeoPandas to interpret the coordinates as geographic locations:

In [ ]:
libraries = gpd.GeoDataFrame(libraries, columns = ['name', 'geometry'])
libraries.head()

## 🗺  Present

When we have geospatial data readily available as GeoDataFrames, we can now map them with Altair.

(There are other mapping libraries for Python, such as [Folium](https://python-visualization.github.io/folium/), that provide additional functionalities. Altair's geovis features are basic, but do provide some variety of techniques and have the benefit to work consistently with the other chart types we covered.)


### Markers on maps

A simple start is placing locations on a base map and adding a bit of further information via tooltips. Let's do this with Austin's schools! First, we can have another look at the attributes:

In [ ]:
schools.head()

We will now create a simple map with markers in the form of an  Altair chart consisting of two layers:

1.   The `neighborhoods` form the lower layer representing their boundaries and the overall geographic shape of Austin
2.   The `schools` are the points of interests that are displayed on top

When putting the two layers together they should actually refer to the same geographic location to make sense. Here the neighborhoods and schools both refer to Austin. Also note that the order when the charts are added together determines the vertical order: first the basemap and then markers on top.

In [ ]:
# 1.  mark_geoshape transparently uses the geometry column
basemap = alt.Chart(neighborhoods).mark_geoshape(
    # add some styling to reduce the salience of the basemap
    fill="lightgray", stroke="darkgray",
).encode(
    tooltip = ['name'],
).properties(width=600, height=600)

# 2.  we use mark_circle to have more control over visual variables
markers = alt.Chart(schools).mark_circle(opacity=1).encode(
    # point latitude & longitude to coordinates in geometry column
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    tooltip=['NAME', 'STREET', 'ZIP'],
)

# combine the two layers
basemap + markers

### Dot density maps

Let's use the open maps data set again, and plot New York's trees. Note, we will have to disable the max rows since there are more than 5,000 trees returned from the query.

In [ ]:
NewYorkCityAreaID = 3600000000 + 175905

# 1. prepare query (and directly include the location lookup)
tree_query = overpassQueryBuilder(
    area=NewYorkCityAreaID,
    elementType='node',
    selector='"natural"="tree"',
    out='body',
    includeGeometry=True
)

# 2. execute query (and give it a bit more time to finish)
tree_data = overpass.query(tree_query, timeout=60)

# 3. get ids and coordinates of trees
tree_locs = [ (tree.id(), tree.geometry()) for tree in tree_data.nodes()]

# 4. create GeoDataFrame
trees = gpd.GeoDataFrame(tree_locs, columns=["id", "geometry"])

trees.head()

In [ ]:
alt.data_transformers.disable_max_rows()
treemap = alt.Chart(trees).mark_circle(
    # reduce the visual presence of each element
    size=5,
    # with a low dot opacity we can use overplotting to indicate densities
    opacity=.25,
    # a natural choice
    color="green"
).encode(
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q'
).properties(width=600, height=600)

treemap

### Choropleth maps

Finally, let's create a geovisualization that uses the fill color of geospatial shapes to encode quantitative data. To illustrate this, we will visualize population densities around the world.

To keep this notebook self-contained, we use the built-in Natural Earth countries dataset that ships with GeoPandas for both country boundaries and basic attributes (population and area).


In [ ]:
# Load country attributes (population + area) from GeoPandas' built-in Natural Earth dataset
import geopandas as gpd

polygons_raw = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
polygons_raw = polygons_raw[polygons_raw["iso_a3"] != "-99"].copy()

# Area in km^2 using an equal-area CRS
area_km2 = polygons_raw.to_crs(epsg=6933).area / 1e6

geonames = polygons_raw.assign(areaInSqKm=area_km2, population=polygons_raw["pop_est"])
geonames = geonames[["name", "iso_a3", "areaInSqKm", "population"]].rename(columns={"iso_a3": "iso alpha3"})
geonames = geonames.set_index("iso alpha3")

geonames.head()

Next we collect the geographic boundaries and `simplify` them a bit, as they have more detail than what we need here:

In [ ]:
# Load country polygons (built-in Natural Earth dataset)
polygons = polygons_raw[["iso_a3", "geometry"]].set_index("iso_a3")
# reduce the complexity of the shapes
polygons.geometry = polygons.geometry.simplify(.1)

polygons.head(20)

As both DataFrames use the three-letter country codes as indices we can join them like this (join uses the index by default, so we don't have to specify what to join on):

In [ ]:
# inner means that we keep only those countries
# for which we have geometric and attribute data
countries = polygons.join(geonames, how='inner')

countries.tail()

Visualizing area or population in a choropleth map, arguably, makes little sense. So let's compute population densities:

In [ ]:
countries["density"] = countries["population"] / countries["areaInSqKm"]
countries.head()

Keep only those countries with valid density value and turn these densities into integers:

In [ ]:
countries = countries[countries['density'].notna()]
countries.density = countries.density.round(0).astype(int)

There is one ‘country’ that is not really one, which is Antarctica. We'll remove this from the list here.

In [ ]:
countries = countries.drop("ATA")

Finally, we draw the chart using Altair's `mark_geoshape()` method. The distribution of densities is highly skewed, due to very small countries with relatively high population numbers, such as Monaco. To spread out the low and high density values we use a logarithmic scale and set the domain between 1 and 1000. Note that the domain has to end in a multiple of the base, which is by default 10.

In [ ]:
alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color('density', scale=alt.Scale(type="log", domain=[1,1000] )),
    tooltip=['name', 'areaInSqKm', 'population', 'density']
).properties(
    width=800,
    height=600
)

The map is shown in the default Mercator projection, which particularly distorts the area sizes of North America, Europe and Russia in contrast to Africa, Southern Asia and parts of South America.

💡 *You can change the projection used above to one that does not distort area sizes as much ([see this list for options](https://vega.github.io/vega-lite/docs/projection.html#projection-types)).*

## Your Turn

**Exploration 1.** Create a visualization with Austin's neighborhoods overlayed with Austin's libraries. The tool tip should provide necessary information to identify each neighborhood and library.

In [ ]:
basemap = alt.Chart(neighborhoods).mark_geoshape(
    fill="lightgray",
    stroke="darkgray"
).encode(
    tooltip=['name:N']
).properties(
    width=600,
    height=600,
    title='Austin Neighborhoods and Libraries'
)

library_markers = alt.Chart(libraries).mark_circle(
    size=100,
    color='red',
    opacity=0.7
).encode(
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    tooltip=['name:N']
)

basemap + library_markers

**Exploration 2.** Do you think the open data source for Austin's libraries is reliable? Why or why not? Answer in the Markdown cell below.

Yes, I think it is reliable for exploration and mapping, but for accuracy/official use, it would be ideal to cross‑check with Austin Public Library or the City of Austin Open Data Portal. OSM is community‑maintained, updated often, and has broad coverage. Also tags (amenity=library) can be inconsistent and include non‑public or duplicate entries. Additionally, closures/new sites can lag; no formal QA or authoritative validation. And also coordinates and names can be noisy.




**Exploration 3.** Add the location of all the trees in Austin (according to Open Street Map) to the visualization you just created in Q1.

In [ ]:
tree_query = overpassQueryBuilder(
    area=AustinAreaID,
    elementType='node',
    selector='"natural"="tree"',
    out='body',
    includeGeometry=True
)

austin_tree_data = overpass.query(tree_query, timeout=60)

austin_tree_locs = [(tree.id(), tree.geometry()) for tree in austin_tree_data.nodes()]

austin_trees = gpd.GeoDataFrame(austin_tree_locs, columns=["id", "geometry"])

alt.data_transformers.disable_max_rows()

basemap = alt.Chart(neighborhoods).mark_geoshape(
    fill="lightgray",
    stroke="darkgray"
).encode(
    tooltip=['name:N']
).properties(
    width=600,
    height=600,
    title='Austin Neighborhoods, Libraries, and Trees'
)

tree_markers = alt.Chart(austin_trees).mark_circle(
    size=3,
    color='green',
    opacity=0.3
).encode(
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q'
)

library_markers = alt.Chart(libraries).mark_circle(
    size=100,
    color='red',
    opacity=0.7
).encode(
    longitude='geometry.coordinates[0]:Q',
    latitude='geometry.coordinates[1]:Q',
    tooltip=['name:N']
)

basemap + tree_markers + library_markers

**Exploration 4.** Create a global population density choropleth with the Albers map projection.

In [ ]:
alt.Chart(countries).mark_geoshape().encode(
    color=alt.Color('density:Q', scale=alt.Scale(type="log", domain=[1, 1000])),
    tooltip=['name:N', 'areaInSqKm:Q', 'population:Q', 'density:Q']
).project(
    type='albers'
).properties(
    width=800,
    height=600,
    title='Global Population Density (Albers Projection)'
)

## Sources

Tutorials & Documentation
- [Specifying Geospatial Data in Altair — Altair 4.1.0 documentation](https://altair-viz.github.io/user_guide/data.html#geospatial-data)
- [GeoPandas](https://geopandas.org)
- [OSMPythonTools](https://github.com/mocnik-science/osm-python-tools)

Data
- [OpenStreetMap](https://www.openstreetmap.org/)
- [GeoNames](https://www.geonames.org)
- [Data.gov](https://catalog.data.gov/dataset/public-school-locations-current)
